# 숙제 
- iris.csv 데이터를 로딩한 다음
- 분류망을 구성하시오.
- parameter tuning을 구현하시오.(pipeline 사용도 함께)

iris 데이터 불러오기

In [1]:
import pandas as pd

iris = pd.read_csv('iris.csv', names=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
iris = iris.iloc[1:,:]
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
1,6.4,2.8,5.6,2.2,2
2,5.0,2.3,3.3,1,1
3,4.9,2.5,4.5,1.7,2
4,4.9,3.1,1.5,0.1,0
5,5.7,3.8,1.7,0.3,0


train, test 데이터로 나눠주기

In [2]:
from sklearn.model_selection import train_test_split
X = iris.iloc[:, 0:4]
Y = iris.iloc[:, 4]

원핫인코딩으로 변환해주기

In [3]:
from keras.utils import np_utils
# np_utils.to_categorical() 로 원핫인코딩으로 변환
y = np_utils.to_categorical(Y, 3)
y[:5]

Using TensorFlow backend.
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_21\Anaconda3\envs

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [4]:
Y[:5]

1    2
2    1
3    2
4    0
5    0
Name: species, dtype: object

최적의 가중치 초기화 & activation함수 찾기

In [6]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [24]:
init_mode =['uniform', 'lecun_uniform', 'normal', 'zero' ,'glorot_normal' ,'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['softmax', 'softplus', 'softsign',' relu','tanh','sigmoid', 'hard_sigmoid','linear']

In [25]:
def create_model(init_mode, activation):
    model = Sequential()
    model.add(Dense(16, input_dim = 4, activation=activation))# 왜 여기는 kernel이 없는가
    model.add(Dense(3, kernel_initializer = init_mode, activation='softmax'))# 왜 softmax
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn = create_model, verbose=0)# 왜 0인가

In [28]:
param_grid = dict(init_mode = init_mode, activation = activation)
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs=-1)
grid_result = grid.fit(X,y)
print("스코어: %f 사용한 조합: %s" %(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with : %r" %(mean,stdev,param))

C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


스코어: 0.480000 사용한 조합: {'activation': 'tanh', 'init_mode': 'he_normal'}
0.306667 (0.057349) with : {'activation': 'softmax', 'init_mode': 'uniform'}
0.366667 (0.165999) with : {'activation': 'softmax', 'init_mode': 'lecun_uniform'}
0.293333 (0.082731) with : {'activation': 'softmax', 'init_mode': 'normal'}
0.373333 (0.154056) with : {'activation': 'softmax', 'init_mode': 'zero'}
0.373333 (0.090431) with : {'activation': 'softmax', 'init_mode': 'glorot_normal'}
0.226667 (0.138884) with : {'activation': 'softmax', 'init_mode': 'glorot_uniform'}
0.226667 (0.118134) with : {'activation': 'softmax', 'init_mode': 'he_normal'}
0.293333 (0.212289) with : {'activation': 'softmax', 'init_mode': 'he_uniform'}
0.280000 (0.033993) with : {'activation': 'softplus', 'init_mode': 'uniform'}
0.353333 (0.093333) with : {'activation': 'softplus', 'init_mode': 'lecun_uniform'}
0.280000 (0.074833) with : {'activation': 'softplus', 'init_mode': 'normal'}
0.253333 (0.054160) with : {'activation': 'softplus', 

최적의 조합은 {'activation': 'tanh', 'init_mode': 'he_normal'}

최적의 epochs, batchsize 찾기

In [32]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim = 4, activation='tanh'))
    model.add(Dense(3, kernel_initializer = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn = create_model, verbose=0)

In [33]:
batch_size = [10,20,40,60,80,100]
epochs = [10,50,100]

param_grid = dict(batch_size = batch_size, epochs = epochs) # compile에서 수행하는 일
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs =-1 )
grid_result = grid.fit(X,y)
print("스코어: %f 사용한 조합: %s"%(grid_result.best_score_, grid_result.best_params_))
mean = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(mean, stds, params):
    print("%f (%f) with : %r" %(mean, stdev, param))

C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


스코어: 0.980000 사용한 조합: {'batch_size': 10, 'epochs': 100}
0.606667 (0.159722) with : {'batch_size': 10, 'epochs': 10}
0.926667 (0.057349) with : {'batch_size': 10, 'epochs': 50}
0.980000 (0.016330) with : {'batch_size': 10, 'epochs': 100}
0.466667 (0.150555) with : {'batch_size': 20, 'epochs': 10}
0.940000 (0.024944) with : {'batch_size': 20, 'epochs': 50}
0.933333 (0.021082) with : {'batch_size': 20, 'epochs': 100}
0.560000 (0.248909) with : {'batch_size': 40, 'epochs': 10}
0.800000 (0.138243) with : {'batch_size': 40, 'epochs': 50}
0.886667 (0.080554) with : {'batch_size': 40, 'epochs': 100}
0.360000 (0.099778) with : {'batch_size': 60, 'epochs': 10}
0.633333 (0.069921) with : {'batch_size': 60, 'epochs': 50}
0.780000 (0.154344) with : {'batch_size': 60, 'epochs': 100}
0.393333 (0.187853) with : {'batch_size': 80, 'epochs': 10}
0.793333 (0.099778) with : {'batch_size': 80, 'epochs': 50}
0.780000 (0.165462) with : {'batch_size': 80, 'epochs': 100}
0.333333 (0.189737) with : {'batch_size

최적의 조합은 {'batch_size': 10, 'epochs': 100}

pipeline사용해 얼마나 정확하게 분류되었는지 확인

In [35]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [16]:
def best_model():
    model = Sequential()
    model.add(Dense(16, input_dim=4, activation='tanh'))
    model.add(Dense(3, kernel_initializer ='he_normal' ,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

In [32]:
seed = 7
np.random.seed(seed)
estimator = KerasClassifier(build_fn = best_model, nb_epoch=100, batch_size = 10, verbose=0)

In [33]:
kfold = KFold(n_splits = 10, random_state=seed)
results = cross_val_score(estimator, X, y, cv = kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(),results.std()))

C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Results: 0.30 (0.18) MSE


In [34]:
results

array([0.        , 0.2       , 0.53333336, 0.26666668, 0.53333336,
       0.26666668, 0.2       , 0.06666667, 0.46666667, 0.46666667])

In [35]:
np.random.seed(seed)
estimators = []
estimators.append(('stanardize', StandardScaler()))
estimators.append(('mip', KerasClassifier(build_fn=best_model, epochs = 50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators) # train, test 데이터의 전처리를 한번에 할 수 있도록 도와줌
kfold = KFold(n_splits = 10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv = kfold)
print("Standardized: %.2f (%.2f MSE)" % (results.mean(), results.std()))

C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Standardized: 0.96 (0.04 MSE)


In [26]:
pipeline.fit(X,y)

Pipeline(memory=None,
         steps=[('stanardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mip',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x0000021AB0FA67C8>)],
         verbose=False)

In [27]:
res = pipeline.predict(X)
res[0]

2